In [14]:
import requests
from bs4 import BeautifulSoup
import csv

In [7]:
url = "https://books.toscrape.com/"

In [8]:
response = requests.get(url)

In [9]:
soup = BeautifulSoup(response.text, "html.parser")

In [10]:
category_links = []
no_pages_list = []
aside = soup.find("aside")
for category in aside.find_all("a")[1:]:
    category_links.append(url + category.get("href")[:-10])
    response = requests.get(url + category.get("href"))
    soup = BeautifulSoup(response.text, "html.parser")
    try:
        no_pages_list.append(int(soup.select_one(".current").get_text().split("of")[1]))
    except:
        no_pages_list.append(1)


In [16]:
books = {
    "Title" : [],
    "Price" : [],
    "Rating" : []
}
for index, link in enumerate(category_links):
    for i in range(no_pages_list[index] + 1):
        response = requests.get(link + f"/page-{i}.html")
        soup = BeautifulSoup(response.text, "html.parser")
        for product in soup.select(".product_pod"):
            rating = product.find("p").get("class")[1]
            books["Title"].append(product.find("h3").get_text())
            books["Rating"].append(rating + " Star")
            books["Price"].append(product.select_one(".price_color").get_text()[1:])

In [17]:
with open("books.csv", "w") as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=["Title", "Price", "Rating"], dialect="excel")
    writer.writeheader()

    for i in range(len(books["Title"])):
        writer.writerow({
            "Title" : books["Title"][i],
            "Price" : books["Price"][i],
            "Rating" : books["Rating"][i]
            })


In [18]:
from openpyxl import Workbook
wb = Workbook()
ws = wb.active

ws.append(["Title", "Price", "Rating"])

for i in range(len(books["Title"])):
        ws.append([
            books["Title"][i],
            books["Price"][i],
            books["Rating"][i]
            ])
wb.save("books.xlsx")